<a href="https://colab.research.google.com/github/junieberry/DL-fromScratch/blob/main/FashionMNIST%20with%20CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd "/content/drive/My Drive/밑시딥"

/content/drive/My Drive/밑시딥


In [ ]:
!git clone https://github.com/WegraLee/deep-learning-from-scratch.git

Cloning into 'deep-learning-from-scratch'...
remote: Enumerating objects: 826, done.
remote: Total 826 (delta 0), reused 0 (delta 0), pack-reused 826
Receiving objects: 100% (826/826), 52.21 MiB | 16.61 MiB/s, done.
Resolving deltas: 100% (477/477), done.


In [ ]:
import tensorflow as tf

import numpy as np
import matplotlib.pyplot as plt

import sys, os
sys.path.append("/content/drive/MyDrive/밑시딥/deep-learning-from-scratch")
from common.layers import *

In [ ]:
fashion_mnist = tf.keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
train_images, test_images = train_images / 255.0, test_images / 255.0

num=10

test_labels=np.eye(num)[np.array(list(test_labels)).reshape(-1)]
train_labels=np.eye(num)[np.array(list(train_labels)).reshape(-1)]

4423680/4422102 [==============================] - 0s 0us/step
[9 2 1 ... 8 1 5]


In [ ]:
x1=np.random.rand(1,3,7,7)
col1=im2col(x1, 5, 5, stride=1, pad=0)
print(col1.shape)

x2=np.random.rand(10,3,7,7)
col2=im2col(x2, 5, 5, stride=1, pad=0)
print(col2.shape)

(9, 75)
(90, 75)


In [44]:
class Convolution:
    def __init__(self, W, b, stride=1, pad=0):
        self.W=W
        self.b=b
        self.stride=stride
        self.pad=pad
    
    def forward(self, x):
        FN, C ,FH, FW=self.W.shape
        N,C,H,W=x.shape
        out_h=int(1+(H+2*self.pad-FH)/self.stride)
        out_w=int(1+(W+2*self.pad-FW)/self.stride)
        
        col=im2col(x, FH, FW, self.stride, self.pad)
        col_W=self.W.reshape(FN,-1).T ##filter
        out=np.dot(col, col_W)+self.b
        
        out=out.reshape(N, out_h, out_w, -1).transpose(0,3,1,2)
        return out
        

In [50]:
class Pooling:
  def __init__(self, pool_h, pool_W, stride=1, pad=0):
    self.pool_h=pool_h
    self.pool_w=pool_W
    self.stride=stride
    self.pad=pad
  
  def forward(self,x):
    N, C, H, W=x.shape
    out_h=int(1+(H-self.pool_h)/self.stride)
    out_w=int(1+(W-self.pool_w)/self.stride)

    ##전개
    col=im2col(x, self.pool_h, self.pool_w,self.stride, self.pad)
    col=col.reshape(-1, self.pool_h, self.pool_w)

    out=np.max(col, axis=1)

    out=out.reshape(N, out_h, out_w, C).transpose(0,3,1,2)

    return out

In [38]:
from collections import OrderedDict

class Convnet:

  def __init__(self, input_dim=(1,28,28),
               conv_param={'filter_num':30, 'filter_size':5,
                           'pad':0,'stride':1},
               hidden_size=100, output_size=10, weight_init=0.01):
    
    filter_num=conv_param['filter_num']
    filter_size=conv_param['filter_size']
    filter_pad=conv_param['pad']
    filter_stride=conv_param['stride']
    intput_size=input_dim[1]
    conv_output_size=(intput_size-filter_size+2*filter_pad)/filter_stride+1
    pool_output_size=int(filter_num*(conv_output_size/2)*(conv_output_size/2))

    self.params={}
    self.params['W1']=weight_init*np.random.rand(filter_num, input_dim[0],filter_size,filter_size)
    self.params['b1']=np.zeros(filter_num)
    self.params['W2']=weight_init*np.random.rand(pool_output_size,hidden_size)
    self.params['b2']=np.zeros(hidden_size)
    self.params['W3']=weight_init*np.random.rand(hidden_size, output_size)
    self.params['b3']=np.zeros(output_size)

    self.layers=OrderedDict()
    self.layers['Conv1']=Convolution(self.params['W1'], self.params['b1'],conv_param['stride'],conv_param['pad'])
    self.layers['Relu1']=Relu()
    self.layers['Pool1']=Pooling(pool_h=2, pool_W=2, stride=2)
    self.layers['Affine1']=Affine(self.params['W2'], self.params['b2'])
    self.layers['Relu2']=Relu()
    self.layers['Affine2']=Affine(self.params['W3'], self.params['b3'])

    self.last_layer=SoftmaxWithLoss()

  def predict(self,x):
    for layer in self.layers.values():
      x=layer.forward(x)
    
    return x
  
  def loss(self, x, t):
    y=self.predict(x)
    return self.last_layer.forward(y,t)
  
  def gradient(self, x, t):
    self.loss(x,t)

    print(x.shape)
    dout=1
    dout=self.last_layer.backward(dout)

    layers=list(self.layers.values())
    layers.reverse()
    for layer in layers:
      dout=layer.backward(dout)
    
    grads={}
    grads['W1'] = self.layers['Conv1'].dW # self.dW = np.dot(self.x.T, dout)
    grads['b1'] = self.layers['Conv1'].db
    grads['W2'] = self.layers['Affine1'].dW
    grads['b2'] = self.layers['Affine1'].db
    grads['W3'] = self.layers['Affine2'].dW
    grads['b3'] = self.layers['Affine2'].db

    return grads

In [34]:
iters=10000
train_size=train_images.shape[0]
batch_size=100
lr=0.01
iter_per_epoch = max(train_size / batch_size, 1)

train_loss_list = []
train_acc_list = []
test_acc_list = []

In [51]:
fashionmodel=Convnet()
temp=10
for i in range(temp):
    batch_mask=np.random.choice(train_size, batch_size)
    x_batch=train_images[batch_mask]
    t_batch=train_labels[batch_mask]

    x_batch=np.reshape(x_batch,(100,1,28,28))

    ## gradient
    grad_backprop=fashionmodel.gradient(x_batch, t_batch)
    
    ## gradient update
    for key in ('W1','b1','W2','b2','W3','b3'):
        fashionmodel.params[key]-=lr*grad_backprop[key]
    
    loss=fashionmodel.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    if i%(iter_per_epoch)==0:
        train_acc=fashionmodel.accuracy(train_images, train_labels)
        test_acc=fashionmodel.accuracy(test_images, test_labels)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        
        print('iters: {:4d}\tTrain acc: {:.5f}\tTest acc: {:.5f}\tLoss: {:f}'.format(i,train_acc,test_acc,loss))
    



print("Train acc : {:.5f}\tTest acc: {:.5f}".format(train_acc, test_acc))


ValueError: ignored